In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import normalize
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
import sys
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import numpy as np

In [ ]:
!pip install --upgrade -q kaggle
!mkdir /root/.kaggle
import json
token = {
    "username":"arsalanhameed",
     "key":"b8257cfda6bc90521ba1a6909ec12c01"
}

with open('/root/.kaggle/kaggle.json', 'w') as config_file:
    json.dump(token, config_file)
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d arsalanhameed/veremi-dos

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Dataset URL: https://www.kaggle.com/datasets/arsalanhameed/veremi-dos
License(s): unknown
100% 881M/884M [00:12<00:00, 103MB/s] 
100% 884M/884M [00:12<00:00, 72.1MB/s]


In [ ]:
import zipfile
zipref=zipfile.ZipFile("/content/veremi-dos.zip",'r')
zipref.extractall()
zipref.close()

!rm -rf /content/veremi-dos.zip

In [ ]:
import pandas as pd
df = pd.read_csv('/content/filtered_data.csv')

In [ ]:
df.head()

,Unnamed: 0,type,rcvTime,pos_0,pos_1,pos_noise_0,pos_noise_1,spd_0,spd_1,spd_noise_0,...,acl_0,acl_1,acl_noise_0,acl_noise_1,hed_0,hed_1,hed_noise_0,hed_noise_1,attack,attack_type
0,2,3,28156.319142,140.514133,944.338854,2.965184,3.066191,-0.346027,4.671520,-0.000469,...,0.333247,-4.486889,0.000448,0.006051,0.256103,0.966649,15.915074,9.636057,1,DoSDisruptive
1,4,2,53612.000000,689.179631,547.143780,3.327547,3.374621,3.887137,-8.732709,0.000090,...,-1.829939,4.111129,0.000028,0.000064,0.360402,-0.932797,5.648109,19.951521,0,DoS
2,10,3,25599.145319,1103.540105,859.459530,3.790750,3.676165,-1.635782,1.483819,0.002357,...,-1.302957,1.181860,0.003797,0.000697,-0.740704,0.671832,22.749374,16.652786,0,DoSDisruptiveSybil
3,13,2,52179.000000,915.381300,856.891979,4.605732,4.441064,-8.882456,-8.557436,-0.009355,...,-0.062381,-0.060092,0.000072,0.000070,-0.885949,-0.463782,32.686788,31.641484,0,DoSDisruptiveSybil
4,29,3,51894.667628,139.785179,876.941507,4.037142,4.037952,0.000010,0.000010,-0.000000,...,0.000006,0.000006,0.000000,0.000000,-0.040485,0.999180,9.325772,9.325772,0,DoSDisruptive


In [ ]:
df = df.iloc[:, 1:]

# Print or further process the DataFrame
print(df)


         type       rcvTime        pos_0       pos_1  pos_noise_0  \
0           3  28156.319142   140.514133  944.338854     2.965184   
1           2  53612.000000   689.179631  547.143780     3.327547   
2           3  25599.145319  1103.540105  859.459530     3.790750   
3           2  52179.000000   915.381300  856.891979     4.605732   
4           3  51894.667628   139.785179  876.941507     4.037142   
...       ...           ...          ...         ...          ...   
6048095     2  53417.000000   396.597546  616.841800     3.390053   
6048096     2  51763.000000   137.175227  988.810612     4.722217   
6048097     3  51567.308877   654.967581  968.820409     4.733575   
6048098     3  28134.663702   884.991967  713.727515     3.712457   
6048099     3  50555.773355   809.926551  758.261899     3.845163   

         pos_noise_1      spd_0      spd_1  spd_noise_0  spd_noise_1  \
0           3.066191  -0.346027   4.671520    -0.000469     0.006336   
1           3.374621   3.88

In [ ]:
df.head()

,type,rcvTime,pos_0,pos_1,pos_noise_0,pos_noise_1,spd_0,spd_1,spd_noise_0,spd_noise_1,acl_0,acl_1,acl_noise_0,acl_noise_1,hed_0,hed_1,hed_noise_0,hed_noise_1,attack,attack_type
0,3,28156.319142,140.514133,944.338854,2.965184,3.066191,-0.346027,4.671520,-0.000469,0.006336,0.333247,-4.486889,0.000448,0.006051,0.256103,0.966649,15.915074,9.636057,1,DoSDisruptive
1,2,53612.000000,689.179631,547.143780,3.327547,3.374621,3.887137,-8.732709,0.000090,-0.000203,-1.829939,4.111129,0.000028,0.000064,0.360402,-0.932797,5.648109,19.951521,0,DoS
2,3,25599.145319,1103.540105,859.459530,3.790750,3.676165,-1.635782,1.483819,0.002357,-0.002137,-1.302957,1.181860,0.003797,0.000697,-0.740704,0.671832,22.749374,16.652786,0,DoSDisruptiveSybil
3,2,52179.000000,915.381300,856.891979,4.605732,4.441064,-8.882456,-8.557436,-0.009355,-0.009012,-0.062381,-0.060092,0.000072,0.000070,-0.885949,-0.463782,32.686788,31.641484,0,DoSDisruptiveSybil
4,3,51894.667628,139.785179,876.941507,4.037142,4.037952,0.000010,0.000010,-0.000000,0.000000,0.000006,0.000006,0.000000,0.000000,-0.040485,0.999180,9.325772,9.325772,0,DoSDisruptive


In [ ]:
# Drop multiple columns
columns_to_drop = ['attack']
df = df.drop(columns=columns_to_drop)


In [ ]:
df.head()

,type,rcvTime,pos_0,pos_1,pos_noise_0,pos_noise_1,spd_0,spd_1,spd_noise_0,spd_noise_1,acl_0,acl_1,acl_noise_0,acl_noise_1,hed_0,hed_1,hed_noise_0,hed_noise_1,attack_type
0,3,28156.319142,140.514133,944.338854,2.965184,3.066191,-0.346027,4.671520,-0.000469,0.006336,0.333247,-4.486889,0.000448,0.006051,0.256103,0.966649,15.915074,9.636057,DoSDisruptive
1,2,53612.000000,689.179631,547.143780,3.327547,3.374621,3.887137,-8.732709,0.000090,-0.000203,-1.829939,4.111129,0.000028,0.000064,0.360402,-0.932797,5.648109,19.951521,DoS
2,3,25599.145319,1103.540105,859.459530,3.790750,3.676165,-1.635782,1.483819,0.002357,-0.002137,-1.302957,1.181860,0.003797,0.000697,-0.740704,0.671832,22.749374,16.652786,DoSDisruptiveSybil
3,2,52179.000000,915.381300,856.891979,4.605732,4.441064,-8.882456,-8.557436,-0.009355,-0.009012,-0.062381,-0.060092,0.000072,0.000070,-0.885949,-0.463782,32.686788,31.641484,DoSDisruptiveSybil
4,3,51894.667628,139.785179,876.941507,4.037142,4.037952,0.000010,0.000010,-0.000000,0.000000,0.000006,0.000006,0.000000,0.000000,-0.040485,0.999180,9.325772,9.325772,DoSDisruptive


In [ ]:
import pandas as pd
import numpy as np

# Assuming 'df' is your DataFrame and 'attack_type' is the name of the column
# Replace 'df' and 'attack_type' with your actual DataFrame and column name

# Define the mapping dictionary for encoding attack types
attack_mapping = {
    'DoS': 0,
    'DoSRandom': 1,
    'DoSDisruptive': 2,
    'DoSDisruptiveSybil': 3,
    'DoSRandomSybil': 4,

}

# Replace attack names with encoded integers
df['encoded_attack_type'] = df['attack_type'].map(attack_mapping)

# Remove infinite or non-finite values from the 'encoded_attack_type' column
df = df.replace([np.inf, -np.inf], np.nan)  # Replace infinite values with NaN
df = df.dropna(subset=['encoded_attack_type'], axis=0)  # Drop rows with NaN values in the 'encoded_attack_type' column

# Convert the 'encoded_attack_type' column to integer data type
df['encoded_attack_type'] = df['encoded_attack_type'].astype(int, errors='ignore')

# Print the DataFrame after encoding and removing non-finite values
print(df[['attack_type', 'encoded_attack_type']])


                attack_type  encoded_attack_type
0             DoSDisruptive                    2
1                       DoS                    0
2        DoSDisruptiveSybil                    3
3        DoSDisruptiveSybil                    3
4             DoSDisruptive                    2
...                     ...                  ...
6048095                 DoS                    0
6048096      DoSRandomSybil                    4
6048097           DoSRandom                    1
6048098           DoSRandom                    1
6048099  DoSDisruptiveSybil                    3

[6048100 rows x 2 columns]


In [ ]:
# Drop multiple columns
columns_to_drop = ['attack_type']
df = df.drop(columns=columns_to_drop)


In [ ]:
# Drop about half of the data points randomly
rows_to_drop = df.sample(frac=0.5).index # Select about 50% of the rows randomly
df_dropped = df.drop(rows_to_drop) # Drop these rows

print(df_dropped)

         type       rcvTime        pos_0        pos_1  pos_noise_0  \
2           3  25599.145319  1103.540105   859.459530     3.790750   
4           3  51894.667628   139.785179   876.941507     4.037142   
8           3  28684.011510   268.738509   217.961930     4.659515   
14          2  27654.000000   214.228006   386.901146     4.714096   
15          3  52470.943226  1100.393006  1423.206605     3.252257   
...       ...           ...          ...          ...          ...   
6048092     3  51850.837278   248.994688   152.464850     4.285076   
6048096     2  51763.000000   137.175227   988.810612     4.722217   
6048097     3  51567.308877   654.967581   968.820409     4.733575   
6048098     3  28134.663702   884.991967   713.727515     3.712457   
6048099     3  50555.773355   809.926551   758.261899     3.845163   

         pos_noise_1      spd_0      spd_1  spd_noise_0  spd_noise_1  \
2           3.676165  -1.635782   1.483819     0.002357    -0.002137   
4           4.0

In [ ]:

# Split data into features (X) and target (y)
X = df_dropped.drop(columns=['encoded_attack_type'])  # Features
y = df_dropped['encoded_attack_type']  # Target variable

In [ ]:
# Basic Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming 'X' is your feature data and 'y' is your target variable
# Replace 'X' and 'y' with your actual feature and target variable names

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Define the Random Forest classifier with verbose=2
rf_classifier = RandomForestClassifier(n_estimators=5, verbose=2)

# Train the Random Forest classifier\
rf_classifier.fit(X_train, y_train)

# Evaluate the model
accuracy = rf_classifier.score(X_test, y_test)
print("Test Accuracy:", accuracy)


building tree 1 of 5
building tree 2 of 5
building tree 3 of 5
building tree 4 of 5
building tree 5 of 5
Test Accuracy: 0.8952861229146344


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# Assuming 'X' is your feature data and 'y' is your target variable
# Replace 'X' and 'y' with your actual feature and target variable names

# Define the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=8, verbose=2)

# Perform 5-fold cross-validation and calculate the accuracy for each fold
cv_scores = cross_val_score(rf_classifier, X, y, cv=5, scoring='accuracy', verbose=2)

# Calculate the average accuracy
average_accuracy = np.mean(cv_scores)
print("5-Fold Cross-Validation Average Accuracy:", average_accuracy)


building tree 1 of 8
building tree 2 of 8
building tree 3 of 8
building tree 4 of 8
building tree 5 of 8
building tree 6 of 8
building tree 7 of 8
building tree 8 of 8
[CV] END .................................................... total time= 4.8min
building tree 1 of 8
building tree 2 of 8
building tree 3 of 8
building tree 4 of 8
building tree 5 of 8
building tree 6 of 8
building tree 7 of 8
building tree 8 of 8
[CV] END .................................................... total time= 5.0min
building tree 1 of 8
building tree 2 of 8
building tree 3 of 8
building tree 4 of 8
building tree 5 of 8
building tree 6 of 8
building tree 7 of 8
building tree 8 of 8
[CV] END .................................................... total time= 4.8min
building tree 1 of 8
building tree 2 of 8
building tree 3 of 8
building tree 4 of 8
building tree 5 of 8
building tree 6 of 8
building tree 7 of 8
building tree 8 of 8
[CV] END .................................................... total time= 4.7min
buil